In [282]:
#coding: utf-8
from pymongo import MongoClient
from py2neo import Graph
import re
from pprint import pprint
client = MongoClient('localhost', 27017)
db = client.istex
coll = db.cambridge

In [283]:
import pandas as pd
import unicodedata

rnsr = pd.read_csv('rnsr.csv', sep=';', encoding='utf-8')
def strip_accents(c):
    s2 = unicodedata.normalize('NFD', c).encode('ascii', 'ignore')  
    return s2
columns = rnsr.columns
rnsr.columns = [strip_accents(c.replace(' ', '_').lower()) for c in columns]

def get_id_from_umr(rnsr, umr):
    f = rnsr[rnsr.iloc[:,2].fillna('UNKNOWN').str.contains(umr)].sort_values('annee')
    if f.groupby('numero_national_de_structure').nunique().shape[0] >= 1:
        return f.sort_values('annee').iloc[0,1]
    else:
        return None

def get_uai_tutelles(rnsr, id_rnsr):
    for uais in rnsr[rnsr['numero_national_de_structure'] == id_rnsr]['uai_des_tutelles'].tolist():
        for uai in str(uais).split(';'):
            yield uai

get_id_from_umr(rnsr, '7639')

u'199719338M'

In [284]:
# Getting authors from French affiliations with UMR
authors = []
aff = re.compile(".*UMR*", re.IGNORECASE)
for i in coll.find({'author.affiliations': aff},{'author':1}):
    for j in i['author']:
        if j.get('affiliations') and 'UMR' in j['affiliations'][0]:
            authors.append({'name': j['name'], 'umr': j['affiliations'][0]})


In [285]:
import tqdm
reg_umr = re.compile("(UMR|CNRS)(| )(\d+)", re.IGNORECASE)
reg_digits = re.compile("\d{4}|\d{3}")
for a in tqdm.tqdm(authors):
    if reg_umr.search(a['umr']):
        a['umr'] = reg_umr.search(a['umr']).group(0).replace('-', ' ')
        if 'UMR ' in a['umr']:
            a['n_umr'] = reg_umr.search(a['umr']).group(3)
            a['id_rnsr'] = get_id_from_umr(rnsr, a['n_umr'])
        elif 'CNRS ' in a['umr']:
            a['umr'] = a['umr'].replace('CNRS', 'UMR')
            n = a['umr'].split(' ')[1]
            a['n_umr'] = n
            a['id_rnsr'] = get_id_from_umr(rnsr, a['umr'])
        elif 'UMR' in a['umr']:
            a['n_umr'] = a['umr'].split('UMR')[1]
            a['id_rnsr'] = get_id_from_umr(rnsr, n)
    elif reg_digits.search(a['umr']):
        a['n_umr'] = reg_digits.search(a['umr']).group(0)
        a['id_rnsr'] = get_id_from_umr(rnsr, a['n_umr'])


100%|██████████| 985/985 [01:28<00:00, 11.18it/s]


In [286]:
from collections import Counter
total_none = Counter([x.get('id_rnsr') for x in authors])[None]
print('Pourcentage de RNSR match avec des chercheurs : %.2f%% sur %s chercheurs' %((100. - float(total_none)*100/len(authors)), len(authors)))

Pourcentage de RNSR match avec des chercheurs : 96.14% sur 985 chercheurs


In [287]:
df_authors = pd.DataFrame(authors)
df_authors.head()

,id_rnsr,n_umr,name,umr
0,199511953U,5509,M. C. JACOB,UMR 5509
1,200012191F,6118,ROMAIN VULLO,UMR 6118
2,199719338M,7639,F. Canova,UMR 7639
3,199719338M,7639,A. Flacco,UMR 7639
4,199719338M,7639,L. Canova,UMR 7639


In [288]:
df_bce = pd.read_csv('uai.csv', delimiter=';', encoding='utf-8')
columns = df_bce.columns
df_bce.columns = [strip_accents(c.replace(' ', '_').lower()) for c in columns]
df_bce.head()

,siren,siret,libelle,sigle,siret_siege,date_de_creation,code_de_categorie,categorie,code_ape,libelle_ape,...,departement,code_de_la_region,region,code_du_pays,pays,geolocalisation,code_de_badge,badge,code_de_la_region_(avant_2016),region_(avant_2016)
0,193500972,NaN,Institut national des sciences appliquées de R...,Insa Rennes,1.935010e+13,1960-06-26,ENS_SUP,Enseignement supérieur,8542Z,Enseignement supérieur,...,Ille-et-Vilaine,53.0,Bretagne,NaN,NaN,"48.1216, -1.63297",anr;brevets;IDFI;LABX;wikipedia,Participant d'un projet financé par l'ANR;Dépo...,53.0,Bretagne
1,193113842,NaN,Université de Toulouse III - Paul Sabatier,NaN,1.931138e+13,1983-01-01,ENS_SUP,Enseignement supérieur,8542Z,Enseignement supérieur,...,Haute-Garonne,76.0,Languedoc-Roussillon - Midi-Pyrénées,NaN,NaN,"43.5608, 1.46936",anr;pcrdt;brevets;EQPX;EQPX;IDFI;LABX;PHUC;wik...,Participant d'un projet financé par l'ANR;Part...,73.0,Midi-Pyrénées
2,195903117,NaN,École nationale supérieure de chimie de Lille,ENSC Lille,1.959031e+13,NaN,ENS_SUP,Enseignement supérieur,8542Z,Enseignement supérieur,...,Nord,32.0,Nord-Pas-de-Calais - Picardie,NaN,NaN,"50.6083, 3.14808",anr;pcrdt;brevets;IEED;wikipedia,Participant d'un projet financé par l'ANR;Part...,31.0,Nord-Pas-de-Calais
3,196244016,NaN,Université d'Artois,NaN,1.962440e+13,1991-11-07,ENS_SUP,Enseignement supérieur,8542Z,Enseignement supérieur,...,Pas-de-Calais,32.0,Nord-Pas-de-Calais - Picardie,NaN,NaN,"50.2849, 2.79104",anr;pcrdt;brevets;IEED;LABX;wikipedia,Participant d'un projet financé par l'ANR;Part...,31.0,Nord-Pas-de-Calais
4,194200937,NaN,École nationale d'ingénieurs de Saint-Étienne,Enise,1.942009e+13,1983-01-01,ENS_SUP,Enseignement supérieur,8542Z,Enseignement supérieur,...,Loire,84.0,Auvergne - Rhône-Alpes,NaN,NaN,"45.4293, 4.42257",anr;pcrdt;brevets;EQPX;wikipedia,Participant d'un projet financé par l'ANR;Part...,82.0,Rhône-Alpes


In [289]:
rnsr_dict
df_bce[['code_uai', 'libelle', 'siren']].to_dict('records')

[{'code_uai': u'0350097R',
  'libelle': u'Institut national des sciences appliqu\xe9es de Rennes',
  'siren': 193500972},
 {'code_uai': u'0311384L',
  'libelle': u'Universit\xe9 de Toulouse III - Paul Sabatier',
  'siren': 193113842},
 {'code_uai': u'0590311T',
  'libelle': u'\xc9cole nationale sup\xe9rieure de chimie de Lille',
  'siren': 195903117},
 {'code_uai': u'0623957P',
  'libelle': u"Universit\xe9 d'Artois",
  'siren': 196244016},
 {'code_uai': u'0420093Y',
  'libelle': u"\xc9cole nationale d'ing\xe9nieurs de Saint-\xc9tienne",
  'siren': 194200937},
 {'code_uai': u'0593559Y',
  'libelle': u'Universit\xe9 Lille 1 - Sciences technologies',
  'siren': 195935598},
 {'code_uai': u'0340132J',
  'libelle': u'\xc9cole nationale sup\xe9rieure d\u2019architecture de Montpellier',
  'siren': 193401320},
 {'code_uai': u'0380141X',
  'libelle': u"\xc9cole nationale sup\xe9rieure d'architecture de Grenoble",
  'siren': 193801412},
 {'code_uai': u'0330199M',
  'libelle': u"\xc9cole d'archit

In [290]:
%%time

from py2neo import Graph
from py2neo import Node
graph = Graph(user='neo4j', password='admin')

authors_dict = {}
rnsr_dict = {}
uai_dict = {}

for etbl in df_bce[['code_uai', 'libelle', 'siren']].to_dict('records'):
    uai_dict[etbl['code_uai']] = Node('Etablissement',
                                          code_uai = etbl['code_uai'],
                                          siren = etbl['siren'],
                                          libelle = etbl['libelle'],
                                      )


for aff in rnsr[['numero_national_de_structure', 'annee', 'tutelles']].to_dict('records'):
    rnsr_dict[aff['numero_national_de_structure']] = Node('Affiliation',
                                                         id_rnsr = aff['numero_national_de_structure'],
                                                         tutelle = aff['tutelles']
                                                         annee = aff['annee']
                                                         )
    
for aut in df_authors.to_dict('records'):
    authors_dict[aut['name']] = Node('Auteur',
                                    name = aut['name'],
                                    )

CPU times: user 10.1 s, sys: 524 ms, total: 10.7 s
Wall time: 10.7 s


In [291]:
%%time

for n in authors_dict.values():
    graph.create(n)
for n in uai_dict.values():
    graph.create(n)
for n in rnsr_dict.values():
    graph.create(n)

CPU times: user 12.3 s, sys: 1.04 s, total: 13.4 s
Wall time: 48 s


In [292]:
%%time

from py2neo import Relationship
links_authors = []
for aut in df_authors.dropna().to_dict('records'):
    a = authors_dict[aut['name']]
    rnsr_node = rnsr_dict[aut['id_rnsr']]
    graph.create(Relationship(a, 'affiliated', rnsr_node))
    for uai_code in get_uai_tutelles(rnsr, aut['id_rnsr']):
        if uai_code != 'nan':
            graph.create(Relationship(rnsr_node, 'tutelle', uai_dict[uai_code]))

CPU times: user 53 s, sys: 3.01 s, total: 56 s
Wall time: 12min 17s
